## Laboratorio 4 - Modelo N-grama
---
- Diego Leiva 21752
- Gustavo Gonzalez 
- Marta Ramirez
- Pablo Orellana 21970

### Implementación de Modelo

#### Librerías

In [1]:
import re
from collections import Counter
import math

#### Lectura de corpus

In [2]:
with open("cr7.txt", "r", encoding="utf-8") as f:
    texto = f.read()
print(texto[:500])

Cristiano Ronaldo forma parte de la leyenda del Real Madrid y siempre será recordado como uno de los grandes símbolos de su historia. 
Fue presentado en el Santiago Bernabéu el 6 de julio de 2009, acompañado por Eusebio y Alfredo Di Stéfano, y desde entonces no paró 
de marcar goles: 451 en 438 partidos oficiales con el Real Madrid (más de uno por encuentro). Además vio puerta en todas las 
competiciones: 312 en la Liga, 105 en la Champions, 22 en la Copa del Rey, 6 en el Mundial de Clubes, 4 en


#### Limpieza | Tokenización | Bigrama | Conteo

In [3]:
# Limpieza del texto
texto_limpio = re.sub(r'[^\w\s]', '', texto.lower())

# Tokenización simple
tokens = texto_limpio.split()

# Bigrama
bigrams = list(zip(tokens[:-1], tokens[1:]))

# Conteo
unigram_counts = Counter(tokens)
bigram_counts = Counter(bigrams)
vocabulario = set(tokens)

print("Primeros tokens:", tokens[:10])
print("Primeros bigramas:", bigrams[:10])

Primeros tokens: ['cristiano', 'ronaldo', 'forma', 'parte', 'de', 'la', 'leyenda', 'del', 'real', 'madrid']
Primeros bigramas: [('cristiano', 'ronaldo'), ('ronaldo', 'forma'), ('forma', 'parte'), ('parte', 'de'), ('de', 'la'), ('la', 'leyenda'), ('leyenda', 'del'), ('del', 'real'), ('real', 'madrid'), ('madrid', 'y')]


#### Cálculo de Entropía y Preplejidad

In [4]:
frase = "cristiano ronaldo forma parte de la leyenda"
tokens_frase = frase.lower().split()
bigrams_frase = list(zip(tokens_frase[:-1], tokens_frase[1:]))

probabilidades = []
for bigrama in bigrams_frase:
    c_bigram = bigram_counts.get(bigrama, 0)
    c_unigram = unigram_counts.get(bigrama[0], 1)
    prob = c_bigram / c_unigram
    probabilidades.append(prob)

entropia = -sum(math.log2(p) for p in probabilidades if p > 0) / len(probabilidades)
perplejidad = 2 ** entropia

print("Probabilidades:", probabilidades)
print("Entropía:", round(entropia, 4))
print("Perplejidad:", round(perplejidad, 4))


Probabilidades: [0.5, 1.0, 1.0, 1.0, 0.25, 0.07142857142857142]
Entropía: 1.1346
Perplejidad: 2.1955


#### Aplicación de Smoothing

In [5]:

# Smoothing
V = len(vocabulario)
k = 0.5

prob_laplace = []
prob_add_k = []
for bigrama in bigrams_frase:
    c_bigram = bigram_counts.get(bigrama, 0)
    c_unigram = unigram_counts.get(bigrama[0], 0)
    laplace = (c_bigram + 1) / (c_unigram + V)
    addk = (c_bigram + k) / (c_unigram + k * V)
    prob_laplace.append(laplace)
    prob_add_k.append(addk)

print("Laplace:", [round(p, 4) for p in prob_laplace])
print("Add-0.5:", [round(p, 4) for p in prob_add_k])


Laplace: [0.016, 0.0161, 0.0161, 0.0161, 0.0476, 0.0146]
Add-0.5: [0.0236, 0.024, 0.024, 0.024, 0.076, 0.0199]
